# Knowledge Representation on the Web -- RDFS tutorial

Here we'll learn the basics of RDFS (RDF Schema) and how to perform basic RDFS reasoning with rdflib (documentation [here](https://rdflib.readthedocs.io/en/stable/)) and owlrl (documentation [here](https://owl-rl.readthedocs.io/en/latest/)).

## Imports

owlrl is a library implementing basic RDFS and OWL reasoning on top of rdflib. We'll install and import its relevant symbols.

In [89]:
import sys
!{sys.executable} -m pip install rdflib owlrl

from rdflib import Graph, RDFS, RDF, URIRef, Literal, Namespace
from owlrl import DeductiveClosure, RDFS_Semantics

fish: Unknown command: truetrue
~/.config/fish/config.fish (line 1): 
truetrue
^
from sourcing file ~/.config/fish/config.fish
	called during startup


## Loading RDFS graphs

Your file `yourRDF.ttl` already contains a basic Knowledge Graph in RDF with some RDFS semantics

First, we are going to add some RDFS semantics, and inspect the graph as-is; this is also called the "asserted graph"

**Exercise 1** 
1. add additional triples using the RDFS semantics: have a look [here](https://www.w3.org/TR/rdf-schema/), and use domain and range, subPropertyOf, and Class, to say more about the instances in your graph
2. load yourRDF graph
3. print the classes in your graph
4. print the properties of a specific class in yourRDF graph
5. print all instances in yourRDF graph (all objects that have a type) 
6. explain what constitutes a vocabulary in RDF

In [90]:
AnimalSpace = Namespace("http://animal.com/")
animalGraph = Graph()
animalGraph.parse("./data/animalGraph.ttl")

animal = AnimalSpace.Animal

person = AnimalSpace.Person
farmer = AnimalSpace.Farmer
daphne = AnimalSpace.Daphne
feeds = AnimalSpace.feeds
feedsGrass = AnimalSpace.feedsGrass
feedsFish = AnimalSpace.feedsFish
cattle = AnimalSpace.Cattle
noCattle = AnimalSpace.Cattle

zebra = AnimalSpace.Zebra
penguin = AnimalSpace.Penguin
pig = AnimalSpace.Pig
chicken = AnimalSpace.Chicken
dog = AnimalSpace.Dog


animalGraph.add((cattle, RDFS.subClassOf, animal))
animalGraph.add((noCattle, RDFS.subClassOf, animal))

animalGraph.add((feeds, RDF.type, RDF.Property))
animalGraph.add((feedsGrass, RDFS.subPropertyOf, feeds))
animalGraph.add((feedsFish, RDFS.subPropertyOf, feeds))
animalGraph.add((person, RDF.type, RDFS.Class))
animalGraph.add((farmer, RDFS.subClassOf, person))
animalGraph.add((daphne, RDF.type, farmer))

animalGraph.add((pig, RDF.type, cattle))
animalGraph.add((penguin, RDF.type, noCattle))
animalGraph.add((chicken, RDF.type, cattle))
animalGraph.add((zebra, RDF.type, noCattle))
animalGraph.add((dog, RDF.type, noCattle))

animalGraph.add((daphne, RDFS.label, Literal("Daphne the farmer")))
animalGraph.add((farmer, feeds, cattle))
animalGraph.add((feeds, RDFS.domain, farmer))
animalGraph.add((feeds, RDFS.range, cattle))

animalGraph.add((daphne, feedsGrass, AnimalSpace.Chicken))
animalGraph.add((daphne, feedsGrass, AnimalSpace.Pig))
animalGraph.add((daphne, feedsFish, AnimalSpace.Penguin))



animalGraph.serialize("./data/newAnimalGraph.ttl")

print(len(animalGraph))

30


## RDFS inferencing

The inference engine in owlrl is triggered by `DeductiveClosure`, which computes the closure of the graph. This requires us to specify under which semantic regime we want to perform the inference (e.g. what kind of rules under the RDFS, OWL, etc. semantics we want the reasoner to produce derivations on). For RDFS semantics we use `RDFS_Semantics` as parameter. See extra options [here](https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.html#module-owlrl)


**Exercise 2**
1. expand the graph through RDFS semantics inference
2. print how many triples the new graph has
3. print out the triples in your new graph and inspect them. 

In [91]:
DeductiveClosure(RDFS_Semantics).expand(animalGraph)

print("Number of triples", len(animalGraph))

animalGraph.serialize("./data/expandedAnimalGraph.ttl")

Number of triples 93


<Graph identifier=N942e628ebece408aadef1993437b1871 (<class 'rdflib.graph.Graph'>)>

## The explicit (asserted) graph vs the implicit (derived) graph, and RDF entailment

Asserted triples are those that are explicitly stated, while derived or inferred triples are those that are implicitly stated through the semantics of RDFS. 

**Exercise 3**

1. Write here code to generate a graph that contains **RDFS derived triples only** from yourRDF Knowledge Graph, not the asserted ones. See a clue on rdflib graph algebra [here](https://rdflib.readthedocs.io/en/stable/merging.html)
2. have a look at the inferred graph. Based on the RDFS semantics, explain for each triple the rule that was used to generate it.
3. Explain the concept RDF entailment, and the types of entailment RDFS can produce


In [92]:
assertedGraph = Graph().parse("./data/animalGraph.ttl")
expandedGraph = Graph().parse("./data/expandedAnimalGraph.ttl")

derivedGraph = expandedGraph - assertedGraph








## Assignment part 2: your own webapplication. 


**Exercise 4**
1. load ingredients.rdf and recipes.rdf in one graph. The graph contains types of individuals and types of relationships between them. Print all the classes and properties in the combined graph with the namespace `ind` and the `wtm` namespace/vocabulary (`http://purl.org/heals/food/`). 

2. extend the `ind` vocabulary (`http://purl.org/heals/ingredient/`) by creating a hierarchy of ingredients (**hint: http://purl.org/heals/ingredient/CoconutMilk rdf:subClassOf http://purl.org/heals/ingredient/PlantMilk), and make these superclasses human readable by giving them labels**) 
3. do the same for the `wtm` vocabulary: add a hierarchy of recipes as well as a hierarchy of properties (**hint: http://purl.org/heals/food/hasCookingTemperature rdf:subPropertyOf ...) 
4. print the entailed triples as we did in the previous exercise
5. give three examples of how RDF semantics could aid the chefs in your restaurant 
    
6. which properties and classes could you add to the `wtm` and `ind` vocabularies to further describe your recipe and ingredient knowledge graph, aiding the chefs in your restaurant?  


In [95]:
ingredientsrecipes = Graph()

ingredientsrecipes.parse("./data/ingredients.rdf")
ingredientsrecipes.parse("./data/recipes.rdf")

# 1. 
ind = Namespace("http://purl.org/heals/ingredient/")
wtm = Namespace("http://purl.org/heals/food/")
# for s, p, o, in ingredientsrecipes.triples((None, None, None)):
#     if wtm in s or ind in s:
#         print(s)
    
#     if wtm in p or ind in p:
#         print(p)
    
#     if wtm in o or ind in o:
#         print(o)

# 2. Creating hierarchy of spices and herbs (which are subclases of seasonings)
spices = ind.Spices
herbs = ind.Herbs
seasoning = ind.Seasoning

herbs_list = ["Tarragon", "Parsley", "Rosemary", "Basil", "Oregano", "Thyme", "Cilantro", ]
spices_list = ["BlackPepper", "Salt"]

for herb in herbs_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{herb}"), RDFS.subClassOf, herbs))

for spice in spices_list:
    ingredientsrecipes.add((URIRef(f"http://purl.org/heals/ingredients/{spice}"), RDFS.subClassOf, spices))


ingredientsrecipes.add((herbs, RDFS.label, Literal("All types of herbs")))
ingredientsrecipes.add((spices, RDFS.label, Literal("All types of spices")))
ingredientsrecipes.add((seasoning, RDFS.subClassOf, wtm.Ingredient))
ingredientsrecipes.add((spices, RDFS.subClassOf, seasoning))
ingredientsrecipes.add((herbs, RDFS.subClassOf, seasoning))
ingredientsrecipes.add((spices, RDFS.label, Literal("All types of seasonings")))

<Graph identifier=N9787a203c1234442806856bd22a09740 (<class 'rdflib.graph.Graph'>)>